In [16]:
%%capture
!pip install folium matplotlib mapclassify

In [17]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import NILs

In [18]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.2009 45.47654, 9.2 45.47696, 9.199..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.207 45.46787, 9.20693 45.46853, 9...."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.2004 45.52848, 9.20028 45.52846, 9..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


# Concatenate all data

In [19]:
# prompt: import all the csv files in the FOLDER_PATH , 'Processed' folder and merge them on ID_NIL, area_sqkm, residenti

import pandas as pd
dfs = []
for filename in os.listdir(os.path.join(FOLDER_PATH, 'Processed')):
  if filename.endswith('.csv'):
    df = pd.read_csv(os.path.join(FOLDER_PATH, 'Processed', filename))
    print(filename)
    df['ID_NIL'] = df['ID_NIL'].astype(str)
    df['area_sqkm'] = df['area_sqkm'].astype(str)
    df = df.drop('NIL',axis=1)
    if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace=True)
    if 'Residenti' in df.columns:
            df.drop(columns=['Residenti'], inplace=True)
    print(df.columns.values)
    dfs.append(df)

merged_df = dfs[0]
for df in dfs[1:]:
  merged_df = pd.merge(merged_df, df, on=['ID_NIL', 'area_sqkm'], how='outer')


charging_stations.csv
['ID_NIL' 'area_sqkm' 'charging_stations' 'charging_stations_per_sqkm']
schools_reachability.csv
['ID_NIL' 'area_sqkm' 'n_schools' 'school_bus_accessibility'
 'urban_transport_accessibility' 'interurban_transport_accessibility'
 'rail_transport_accessibility' 'private_transport_accessibility'
 'disable_transport_accessability']
sports_centers.csv
['ID_NIL' 'area_sqkm' 'n_sport_centers']
beds_and_nursing_homes.csv
['ID_NIL' 'area_sqkm' 'n_schools' 'ordinary_hospitalization'
 'payment_hospitalization' 'day_hospital' 'day_surgery' 'total_beds']
remediable_buildings.csv
['ID_NIL' 'area_sqkm' 'remediable_buildings']
green_spaces.csv
['ID_NIL' 'area_sqkm' 'green_spaces_sqkm' 'green_spaces_ratio'
 'green_space_per_person']
bike_lanes.csv
['ID_NIL' 'area_sqkm' 'bike_lane_km']
parkings_grouped.csv
['ID_NIL' 'area_sqkm' 'parking_sqkm']
stops_grouped.csv
['ID_NIL' 'area_sqkm' 'count_underground' 'count_surface'
 'underground_stops_per_resident' 'surface_stops_per_resident'
 

In [20]:
merged_df['geometry']

,geometry
0,POLYGON ((9.194822062744754 45.472010572801985...
1,"POLYGON ((9.21835697737147 45.49970313665964, ..."
2,"POLYGON ((9.201164607426154 45.4896311980832, ..."
3,"POLYGON ((9.18489219966993 45.50554396373852, ..."
4,"POLYGON ((9.212198638860967 45.50970920856589,..."
...,...
83,"POLYGON ((9.217107242057669 45.4318680705707, ..."
84,"POLYGON ((9.138855346957909 45.4285488853788, ..."
85,POLYGON ((9.046873591820898 45.462761504113615...
86,"POLYGON ((9.073090621959555 45.50603636678944,..."


In [21]:
merged_df.columns

Index(['ID_NIL', 'area_sqkm', 'charging_stations',
       'charging_stations_per_sqkm', 'n_schools_x', 'school_bus_accessibility',
       'urban_transport_accessibility', 'interurban_transport_accessibility',
       'rail_transport_accessibility', 'private_transport_accessibility',
       'disable_transport_accessability', 'n_sport_centers', 'n_schools_y',
       'ordinary_hospitalization', 'payment_hospitalization', 'day_hospital',
       'day_surgery', 'total_beds', 'remediable_buildings',
       'green_spaces_sqkm', 'green_spaces_ratio', 'green_space_per_person',
       'bike_lane_km', 'parking_sqkm', 'count_underground', 'count_surface',
       'underground_stops_per_resident', 'surface_stops_per_resident',
       'underground_stops_per_sqkm', 'surface_stops_per_sqkm', 'geometry',
       '0-2 anni', '11-13 anni', '14-18 anni', '19-24 anni', '25-34 anni',
       '3-5 anni', '35-44 anni', '45-54 anni', '55-64 anni', '6-10 anni',
       '65-74 anni', '75-84 anni', '85 anni e più', 'pr

In [22]:
merged_df['ID_NIL'] = merged_df['ID_NIL'].astype(str)
nil['ID_NIL'] = nil['ID_NIL'].astype(str)
merged_df = merged_df.drop('geometry', axis=1)


In [25]:
# prompt: merge merged_df and nil on 'ID_NIL' column

merged_df = pd.merge(merged_df, nil[['ID_NIL', 'NIL' ,'Residenti', 'geometry']], on='ID_NIL', how='left')
merged_df

,ID_NIL,area_sqkm,charging_stations,charging_stations_per_sqkm,n_schools_x,school_bus_accessibility,urban_transport_accessibility,interurban_transport_accessibility,rail_transport_accessibility,private_transport_accessibility,...,public_schools_per_sqkm,public_toilets,public_toilets_per_sqkm,bike_slots,bike_slots_per_sqkm,roads_km,walkways_km,NIL,Residenti,geometry
0,1,2.34161556588591,22.0,9.395223,10,3.0,7.0,3.0,3.0,7.0,...,0.427056,18,7.687000,79,33.737391,39.587619,16.259897,duomo,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,10,1.5560187497830058,15.0,9.639987,2,0.0,2.0,2.0,2.0,2.0,...,1.927997,12,7.711989,10,6.426658,31.257577,1.052164,stazione centrale - ponte seveso,18410,"POLYGON ((9.21836 45.4997, 9.21736 45.50008, 9..."
2,11,1.3228871157647772,2.0,1.511845,7,3.0,3.0,0.0,2.0,6.0,...,0.755922,2,1.511845,3,2.267767,25.826316,1.085650,isola,22641,"POLYGON ((9.20116 45.48963, 9.19881 45.49081, ..."
3,12,1.6749176277225284,3.0,1.791133,9,1.0,6.0,4.0,1.0,6.0,...,0.000000,3,1.791133,1,0.597044,36.406028,0.333126,maciachini - maggiolina,26097,"POLYGON ((9.18489 45.50554, 9.18378 45.50571, ..."
4,13,1.7686026100672578,2.0,1.130836,5,0.0,4.0,1.0,0.0,3.0,...,0.000000,2,1.130836,2,1.130836,19.614379,0.914580,greco - segnano,16137,"POLYGON ((9.2122 45.50971, 9.21199 45.50967, 9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,85,13.73384050477448,0.0,0.000000,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,1,0.072813,1,0.072813,32.589335,0.833203,parco delle abbazie,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
84,86,3.6231494136433935,0.0,0.000000,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,1,0.276003,2,0.552006,8.605110,0.000000,parco dei navigli,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
85,87,5.840941710883686,0.0,0.000000,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,1,0.171205,1,0.171205,23.416219,0.000000,assiano,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."
86,88,7.834006115163006,0.0,0.000000,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,1,0.127649,2,0.255297,29.264848,0.000000,parco bosco in citta',690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ..."


In [26]:
merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

In [27]:
merged_df.to_file(os.path.join(FOLDER_PATH, 'Processed','merged_df.geojson'), driver='GeoJSON')

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: NaN of Infinity value found. Skipped
  ogr_write(
